In [41]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.models.rnn import rnn, rnn_cell
%matplotlib inline

mnist = input_data.read_data_sets('data/', one_hot=True)
trainimgs   = mnist.train.images
trainlabels = mnist.train.labels
testimgs   = mnist.test.images
testlabels  = mnist.test.labels


ntrain = trainimgs.shape[0] # numbe of training examples

nclasses = trainlabels.shape[1] # number of output classes

dim    = trainimgs.shape[1] # dimension of input

ntest  = testimgs.shape[0] # number of test examples

print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("%d classes" % (nclasses))

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
55000 train images loaded
10000 test images loaded
784 dimensional input
10 classes


In [42]:
# provided code from LSTM_MNIST.ipynb

# Recurrent neural network 
diminput  = 28
dimhidden = 128
dimoutput = nclasses
nsteps    = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput, dimhidden])), 
    'out': tf.Variable(tf.random_normal([dimhidden, dimoutput]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}
def _RNN(_X, _istate, _W, _b, _nsteps, _name):
    # 1. Permute input from [batchsize, nsteps, diminput] => [nsteps, batchsize, diminput]
    _X = tf.transpose(_X, [1, 0, 2])
    # 2. Reshape input to [nsteps*batchsize, diminput] 
    _X = tf.reshape(_X, [-1, diminput])
    # 3. Input layer => Hidden layer
    _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
    # 4. Splite data to 'nsteps' chunks. An i-th chunck indicates i-th batch data 
    _Hsplit = tf.split(0, _nsteps, _H) 
    # 5. Get LSTM's final output (_O) and state (_S)
    #    Both _O and _S consist of 'batchsize' elements
    with tf.variable_scope(_name, reuse=True): # <= NEW reuse = TRUE, to get rid of naming collision
        lstm_cell = rnn_cell.BasicLSTMCell(dimhidden, forget_bias=1.0)
        _LSTM_O, _LSTM_S = rnn.rnn(lstm_cell, _Hsplit, initial_state=_istate)
    # 6. Output
    _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']    
    # Return! 
    return {
        'X': _X, 'H': _H, 'Hsplit': _Hsplit,
        'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O 
    }

In [43]:
# 1. • Complete implementation of MNIST training with LSTM model

# define input x and teacher y
x  = tf.placeholder(tf.float32, [None, nsteps, diminput])
y  = tf.placeholder(tf.float32, [None, nclasses])

# 1.1. • Training with initialized weights and zero 2x128 initial states (state & cell)
stateAndCell = tf.placeholder(tf.float32, [None, 2 * dimhidden])

# 1.2. • Use the provided RNN model
customRNN  = _RNN(x, stateAndCell, weights, biases, nsteps, 'custom')

# Parameters
learning_rate   = 0.001
training_epochs = 7 # <= gives 0.978 test accuracy
batch_size      = 128
display_step    = 1

# define cost function based on output of rnn and teach value y
# Index '0' refers to output of recurrent neural networks
cost   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(customRNN['O'], y))
optimizer   = tf.train.AdamOptimizer(learning_rate).minimize(cost)
accuracy   = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(customRNN['O'], 1), tf.argmax(y, 1)), tf.float32))

# init all variables
init   = tf.initialize_all_variables()

# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain / batch_size)
    # Loop over all batches
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput)) # reshape images in thr batch to ? x 28 x 28
        
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, stateAndCell: np.zeros((batch_size, 2 * dimhidden))})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, 
                                              stateAndCell: np.zeros((batch_size, 2 * dimhidden))}) / num_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, 
                                                  stateAndCell: np.zeros((batch_size, 2 * dimhidden))})
        print ("Training accuracy: %.3f" % (train_acc))
        testimgs = testimgs.reshape((ntest, nsteps, diminput)) # reshape test images to ? x 28 x 28
        test_acc = sess.run(accuracy, feed_dict={x: testimgs, y: testlabels, 
                                                 stateAndCell: np.zeros((ntest, 2 * dimhidden))})
        print ("Test accuracy: %.3f" % (test_acc))

print ("DONE!")

Epoch: 000/007 cost: 0.561759554
Training accuracy: 0.992
Test accuracy: 0.909
Epoch: 001/007 cost: 0.148232929
Training accuracy: 0.961
Test accuracy: 0.943
Epoch: 002/007 cost: 0.090823532
Training accuracy: 0.938
Test accuracy: 0.967
Epoch: 003/007 cost: 0.067435335
Training accuracy: 1.000
Test accuracy: 0.964
Epoch: 004/007 cost: 0.054738691
Training accuracy: 0.984
Test accuracy: 0.978
Epoch: 005/007 cost: 0.045760867
Training accuracy: 0.992
Test accuracy: 0.975
Epoch: 006/007 cost: 0.037110288
Training accuracy: 0.984
Test accuracy: 0.977
DONE!


In [44]:
# 2. • Test accuracy with smaller number of steps

# 2.1. • 28 steps mean using full image (28 x 28)
smallerSteps = 20 # number of steps <= 28

# 2.2. • You can use smaller steps to predict test image label. It means the
# trained LSTM model works even the input image is truncated with a
# certain level of accuracy
testimgs = testimgs.reshape((ntest, nsteps, diminput)) # reshape test images to ? x 28 x 28
truncated_imgs = np.zeros(testimgs.shape)
truncated_imgs[:, 28 - smallerSteps:] = testimgs[:, :smallerSteps, :]

# 2.3. • Show the accuracy drop-out according to the number of steps.
test_acc = sess.run(accuracy, feed_dict={x: truncated_imgs, y: testlabels, 
                                     stateAndCell: np.zeros((ntest, 2 * dimhidden))})
print ("Test accuracy is %.3f, when using %d steps: " % (test_acc, smallerSteps))

Test accuracy is 0.489, when using 20 steps: 
